In [1]:
#Write a class (e.g. named UserItemData) in which you will store the read data. 
#I suggest that you have a path argument and three optional arguments in the class constructor: 
 #   from_date (from which date to read data), to_date (by which date to read data), 
  #  and min_ratings (minimum number of ratings a movie should have). Add a method to the 
   # class that tells you how many ratings it has read. I suggest that you add a method that saves the 
    #read data or read the data using the pickle library
import pandas as pd
import datetime

class UserItemData():
    def __init__(self, *args):
        # if args is 1 - path
        if len(args) == 1:
            self.df = pd.read_csv(args[0], sep='\t')
        elif (len(args) == 2):
            self.path = args[0]
            self.min_ratings = args[1]
            self.df1 = pd.read_csv(self.path, sep='\t')
            #movies count filter
            self.df2 = self.df1.groupby('movieID')["rating"].count().reset_index(name="count")
            self.movies_enough_ratings = self.df2["movieID"][self.df2["count"] >= self.min_ratings]
            self.movies_list = self.movies_enough_ratings.to_list()
            self.df = self.df1[self.df1['movieID'].isin(self.movies_list)]
        else:
            self.path = args[0]
            self.from_datetime = datetime.datetime.strptime(args[1], "%d.%m.%Y")
            self.to_datetime = datetime.datetime.strptime(args[2], "%d.%m.%Y")
            self.min_ratings = args[3]
            self.df1 = pd.read_csv(self.path, sep='\t')
            #computed date
            self.df1["full_date"] = self.df1["date_day"].astype(str) + "." + self.df1["date_month"].astype(str) + "." + self.df1["date_year"].astype(str)
            #dates filter
            self.cond = self.df1.loc[(pd.to_datetime(self.df1['full_date']) >= self.from_datetime) & (pd.to_datetime(self.df1['full_date']) <= self.to_datetime)]
            #movies count filter
            self.df2 = self.cond.groupby('movieID')["rating"].count().reset_index(name="count")
            self.movies_enough_ratings = self.df2["movieID"][self.df2["count"] >= self.min_ratings]
            self.movies_list = self.movies_enough_ratings.to_list()
            self.df = self.cond[self.cond['movieID'].isin(self.movies_list)]
    def nratings(self):
        return len(self.df.index)

uim1 = UserItemData('data/user_ratedmovies.dat', "12.01.2007", "16.02.2008", 1000)
uim333 = UserItemData('data/user_ratedmovies.dat', 1000)
uim = UserItemData('data/user_ratedmovies.dat')
print(uim1.nratings())
print(list(uim1.df1.columns))
#print(len(uim333.df1.index))
#print(lolo1.df.head(5))
#print(lolo1.cond)
#print(lolo1.td_list[2])
#print(uim1.nratings())    
#print(uim1.df)

0
['userID', 'movieID', 'rating', 'date_day', 'date_month', 'date_year', 'date_hour', 'date_minute', 'date_second', 'full_date']


In [2]:
#Write a class that reads the movie file and has a get_title(movieID) method that
#returns its title for a given movie ID. Example:

#md = MovieData('data/movies.dat')
#print(md.get_title(1))
#Output:
#Toy story
import pandas as pd
class MovieData():
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(self.path, sep='\t')
    def get_title(self, num):
        return ((self.df['title'])[self.df.id == num].to_string(index=False))
        #return self.df['title'][self.df.id == num]
    
md1 = MovieData("data/movies.dat")
print(md1.get_title(65011))

Zona Zamfirova


In [14]:
#The word "predictor" will be used to denote classes that, in a certain way, assess for a certain user 
#the rating he would give to the movies or the products available to him. 
#These classes will have a fit(self, X) method, where X is of type UserItemData, 
#and a predict(self, user_id) method, where user_id is the user ID. 
#We will use the fit method to learn the model, and the predict to calculate the 
#recommended values for a given user. Recommended values can be e.g. returned as a dictionary.

#Random predictor (+)
#Write a RandomPredictor class that accepts a minimum and maximum rating in the constructor, 
#and the predict method returns a random value (between min and max) for each product.

#md = MovieData('data/movies.dat')
#uim = UserItemData('data/user_ratedmovies.dat') 
#rp = RandomPredictor(1, 5) rp.fit(uim) 
#pred = rp.predict(78) 
#print(type(pred)) 
#items = [1, 3, 20, 50, 100] 
#for item in items: 
#    print("Movie: {}, score: {}".format(md.get_title(item), pred[item]))
import random
#1 : 10198
#RATED MOVIES USER HAS NOT SEEN YET BY DEFAULT
class RandomPredictor():
    def __init__(self, min_num, max_num):
        self.min_num = min_num
        self.max_num = max_num
    def fit(self, frame):
        self.df = frame.df
        #rec_seen -> if you want to predict movie ratings of movies user has or has not 
    def predict(self, user_id, n, rec_seen):
        #all movieid's
        self.all_movies_df = pd.read_csv("data/movies.dat", sep='\t')
        self.movies = self.all_movies_df['id'].to_list()
        #fill random rates to movies that "user_id" havent watched yet
        if (rec_seen):
            return self.predict_seen(user_id)
        else:
            return self.predict_unseen(user_id)
    def predict_unseen(self,user_id):
        self.user_rates = self.df[self.df['userID'] == user_id]
        self.movie = self.user_rates['movieID'].to_list()
        #otherwise insert movies table to get number of movies
        #all movies
        self.full_list = self.df['movieID'].to_list()
        self.to_watch = self.get_unwatched_movies(self.full_list, self.movie)
        
        #fill random rates to movies that "user_id" havent watched yet
        dicti = {}
        for i in self.to_watch:
                dicti[i] = random.randint(self.min_num,self.max_num)
        return dicti
    def predict_seen(self, user_id):
        self.user_rates = self.df[self.df['userID'] == user_id]
        self.movie = self.user_rates['movieID'].to_list()
        dicti = {}
        for i in self.movie:
                dicti[i] = random.randint(self.min_num,self.max_num)
        return dicti
    #enter all movies so we can find out which user havent seen yet
    def movies_counter(self, movies_frame):
        self.num_movies = len(self.df.index)
    def get_unwatched_movies(self, all_movies, seen):
        new_movies = all_movies
        for s in seen:
            if s in all_movies:
                new_movies.remove(s)
        return new_movies


md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat') 
rp = RandomPredictor(1, 5) 
rp.fit(uim) 
rec_seen = False
pred = rp.predict(78, 10, rec_seen) 
print(type(pred)) 
items = [1, 3, 20, 50, 100]
#prints predictions for movies with some scores
for item in items: 
    print("Movie: {}, score: {}".format(md.get_title(item), pred[item]))
    


<class 'dict'>
Movie: Toy story, score: 3
Movie: Grumpy Old Men, score: 3
Movie: Money Train, score: 3
Movie: The Usual Suspects, score: 1
Movie: City Hall, score: 3


In [15]:
#Recommendation (+)
#Write a class Recommender. The class should accept the predictor in the constructor and have two methods: 
#fit(self, X) and recommend(self, userID, n = 10, rec_seen = True). 
#The first method has the same function as in the predictor, while the second method returns an edited list of 
#recommended products for the userID user. The parameter n determines the number of recommended movies, 
#and with rec_seen we determine whether we want the already watched movies (those to which the user has 
#already given a rating) or not.
import random

class Recommender():
    def __init__(self, predictor):
        self.predictor = predictor
    def fit(self, x):
        self.rates_frame = x.df
    def recommend(self, userID ,n ,rec_seen):
        if (rec_seen):
            #ratings movies user already seen
            self.seen_movies_dict = self.predictor.predict(userID, n, rec_seen)
            len_dict = len(self.seen_movies_dict) 
            remove = len_dict - n 
            if n <= len(self.seen_movies_dict) - 2:  
                #limit to n movies
                cnt = 0
                for i in range(1, remove):
                    #remove random movies
                    self.seen_movies_dict.pop(list(self.seen_movies_dict)[random.randint(1, (len - 1) - cnt)])
                    cnt = cnt + 1
                #sort by rates
                #return self.sort_dict(self.seen_movies_dict)
                return self.seen_movies_dict
            else:
                #return self.sort_dict(self.seen_movies_dict)
                return self.seen_movies_dict
        else:
            #movies not seen yet - > dict
            self.not_seen = self.predictor.predict(userID,n, rec_seen)
            len_dict = len(self.not_seen)
            remove = len_dict - n 
            #limit to n movies
            if n <= len(self.not_seen) - 2: 
                cnt = 0
                for i in range(1, remove):
                    self.not_seen.pop(list(self.not_seen)[random.randint(1, (len_dict - 1) - cnt)])
                    cnt = cnt + 1
                #sort by rates
                return self.sort_dict(self.not_seen)
            else:
                return self.sort_dict(self.seen_movies_dict)
    def sort_dict(self, dicti):
        tuples = dicti.items()
        sorted_tuples = sorted(tuples, key=self.sort_second, reverse=True)
        new_dict = {}
        for i in sorted_tuples:
            new_dict[i[0]] = i[1]
        return new_dict
    def sort_second(self, element):
        return element[1]

        
mdd = MovieData('data/movies.dat') 
uim_r = UserItemData('data/user_ratedmovies.dat') 
rp_r = RandomPredictor(1, 5)
rp_r.fit(uim_r) 
rec = Recommender(rp_r) 
rec.fit(uim_r) 
rec_items = rec.recommend(78, n=5, rec_seen=False) 
for idmovie, val in rec_items.items(): 
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val)) 

Movie: Grumpy Old Men, score: 5
Movie: Going in Style, score: 5
Movie: Whose Life Is It Anyway?, score: 3
Movie: Mr. & Mrs. Smith, score: 2
Movie: A Good Year, score: 2
Movie: Bis ans Ende der Welt, score: 1


Average predictor (+)
Write the class AveragePredictor, which takes the parameter b in the constructor, where b >= 0. In the fit method, calculate the average for each movie according to the formula avg = (vs + b * g_avg) / (n + b), where:

vs is the sum of all ratings for this movie,  
n is the number of scores this movie received,  
g_avg is the average of all movies,  
b is the parameter of the formula for the average. If b=0, it is a simple average  
If we use AveragePredictor(b=0) in the recommender and the rest is the same as in the previous section, we get:  

Movie: Sonnenallee, score: 5.0  
Movie: Vals Im Bashir, score: 5.0  
Movie: Britannia Hospital, score: 5.0  
Movie: Il mio viaggio in Italia, score: 5.0  
Movie: Shu dan long wei, score: 5.0 
However, if we use AveragePredictor(b=100), the values change: 

Movie: The Usual Suspects, score: 4.225944245560473  
Movie: The Godfather: Part II, score: 4.146907937910189  
Movie: Cidade de Deus, score: 4.116538340205236  
Movie: The Dark Knight, score: 4.10413904093503  
Movie: 12 Angry Men, score: 4.103639627096175 

In [16]:
import math
class AveragePredictor():
    def __init__(self, b):
        self.b = b
    def fit(self, frame):
        #movies table to get all the movie ids
        self.movies_object = MovieData("data/movies.dat")
        #movie id's
        self.movies = self.movies_object.df['id'].to_list()
        #UserItemData frame that we insert
        self.df = frame.df
        #sum of all ratings for each movie
        self.sums = self.df.groupby('movieID')["rating"].sum().reset_index(name="sum")
        #num of ratings per each movie
        self.counts = self.df.groupby('movieID')["rating"].count().reset_index(name="count")
        #avg rate
        self.all_average = self.df["rating"].mean()
        dicti = {}
        for i in self.movies:
            #(vs + b * g_avg) / (n + b)
            if (i == 53355):
                print("found")
            summ = self.sums['sum'][self.sums["movieID"] == i ]
            sum_1 = float(summ.max())
            cnt = self.counts['count'][self.counts["movieID"] == i ]
            cnt_1 = float(cnt.max())
            #insert in dict  ->> movieID : average
            if (self.b == 0):
                score = (sum_1) / (cnt_1)
            else:
                score = (sum_1 + self.b * self.all_average) / (cnt_1 * self.b)
            dicti[i] = score
        #sort dictionary
        return self.sort_dict(dicti)
    def sort_dict(self, dicti): #descending
        tuples = dicti.items()
        sorted_tuples = sorted(tuples, key=self.sort_second, reverse=True)
        new_dict = {}
        for i in sorted_tuples:
            if (math.isnan(i[1])):
                continue
            new_dict[i[0]] = i[1]
        return new_dict
    def sort_second(self, element):
        return element[1]



In [17]:
uim_avg = UserItemData('data/user_ratedmovies.dat')
ap = AveragePredictor(0)
avg_calc = ap.fit(uim_avg)
#prints predictions for movies with some scores
#movies get right scores, just the order is not correct
#g_avg might be calculated incorecctly
for item in avg_calc: 
    print("Movie: {}, score: {}".format(md.get_title(item), avg_calc[item]))

    

found
Movie: Brother Minister: The Assassination of Malcolm X, score: 5.0
Movie: Synthetic Pleasures, score: 5.0
Movie: Storefront Hitchcock, score: 5.0
Movie: Shanghai Ghetto, score: 5.0
Movie: Finnegan Begin Again, score: 5.0
Movie: Message to Love: The Isle of Wight Festival, score: 4.625
Movie: Sonic Outlaws, score: 4.5
Movie: From the Journals of Jean Seberg, score: 4.5
Movie: Etz Hadomim Tafus, score: 4.5
Movie: Il testimone dello sposo, score: 4.5
Movie: A Century of Cinema, score: 4.5
Movie: A Little Bit of Soul, score: 4.5
Movie: Lan yu, score: 4.5
Movie: The Shawshank Redemption, score: 4.365371269951423
Movie: Paid in Full, score: 4.333333333333333
Movie: The Life and Death of Colonel Blimp, score: 4.325
Movie: Rekopis znaleziony w Saragossie, score: 4.3
Movie: The Usual Suspects, score: 4.29064039408867
Movie: Rude Boy, score: 4.25
Movie: Pulp Fiction, score: 4.238451528952505
Movie: Schindler's List, score: 4.174786324786325
Movie: The Matrix, score: 4.173952095808383
Movi

Movie: Bastard Out of Carolina, score: 3.3846153846153846
Movie: Flesh and Bone, score: 3.3823529411764706
Movie: The Birdcage, score: 3.380855397148676
Movie: The Good Girl, score: 3.378787878787879
Movie: Notting Hill, score: 3.3771929824561404
Movie: The Basketball Diaries, score: 3.3761061946902653
Movie: Four Rooms, score: 3.375
Movie: Across the Sea of Time, score: 3.375
Movie: The Reckless Moment, score: 3.375
Movie: Crooklyn, score: 3.375
Movie: Love and Other Catastrophes, score: 3.375
Movie: Xing xing wang, score: 3.375
Movie: The Ghost of Frankenstein, score: 3.375
Movie: Legends of the Fall, score: 3.3713826366559485
Movie: Ai no kor�da, score: 3.369565217391304
Movie: La promesse, score: 3.3666666666666667
Movie: Crimes of the Heart, score: 3.3666666666666667
Movie: Kalifornia, score: 3.3664383561643834
Movie: Romeo Is Bleeding, score: 3.3658536585365852
Movie: Mommie Dearest, score: 3.3636363636363638
Movie: Frankenstein Meets the Wolf Man, score: 3.3636363636363638
Movie

Movie: Newsies, score: 2.9038461538461537
Movie: Star Wars: Episode I - The Phantom Menace, score: 2.9016754850088184
Movie: Down in the Delta, score: 2.9
Movie: Romy and Michele's High School Reunion, score: 2.8975265017667846
Movie: Addams Family Values, score: 2.896103896103896
Movie: The Preacher's Wife, score: 2.895348837209302
Movie: Kama Sutra: A Tale of Love, score: 2.888888888888889
Movie: Desert Blue, score: 2.888888888888889
Movie: Up Close & Personal, score: 2.8859649122807016
Movie: Demolition Man, score: 2.8847517730496453
Movie: The Cure, score: 2.875
Movie: Tales from the Crypt: Demon Knight, score: 2.875
Movie: Breakfast of Champions, score: 2.875
Movie: Fathom, score: 2.875
Movie: Grumpy Old Men, score: 2.873015873015873
Movie: Poetic Justice, score: 2.8636363636363638
Movie: Biggie and Tupac, score: 2.8636363636363638
Movie: Vibes, score: 2.857142857142857
Movie: Jumpin' Jack Flash, score: 2.85
Movie: Mr. Jones, score: 2.8421052631578947
Movie: Radioland Murders, sco

Movie: Celtic Pride, score: 2.289473684210526
Movie: Sunset Park, score: 2.25
Movie: Solo, score: 2.2291666666666665
Movie: Boy A, score: 2.2222222222222223
Movie: Galaxy Quest, score: 2.2051282051282053
Movie: Operation Dumbo Drop, score: 2.2
Movie: Flipper, score: 2.1444444444444444
Movie: Bloodsport 2, score: 1.9705882352941178
Movie: All Dogs Go to Heaven 2, score: 1.951219512195122
Movie: A Thin Line Between Love and Hate, score: 1.75
Movie: Das Superweib, score: 1.75
Movie: Theodore Rex, score: 1.5
Movie: 301, 302, score: 1.5
Movie: Delta of Venus, score: 1.5
Movie: Billy's Holiday, score: 0.5
Movie: Dr. Strangelove or: How I Learned to Stop Worry..., score: 4.225059382422803
Movie: Wallace and Gromit in A Close Shave, score: 4.093096234309623
Movie: Trainspotting, score: 4.004624277456648
Movie: The Riches, score: 4.0
Movie: Getting Away with Murder, score: 4.0
Movie: Brothers in Trouble, score: 4.0
Movie: The Last of the High Kings, score: 4.0
Movie: Marlene Dietrich: Shadow an

Movie: Some Kind of Wonderful, score: 3.4411764705882355
Movie: The English Patient, score: 3.4297520661157024
Movie: Romeo + Juliet, score: 3.423076923076923
Movie: Grace of My Heart, score: 3.409090909090909
Movie: The Ghost and the Darkness, score: 3.396694214876033
Movie: Picnic, score: 3.388888888888889
Movie: Carrie, score: 3.383076923076923
Movie: The Great Race, score: 3.380952380952381
Movie: Up in Smoke, score: 3.380434782608696
Movie: The Scarlet Letter, score: 3.375
Movie: Bad Taste, score: 3.3461538461538463
Movie: Invasion of the Body Snatchers, score: 3.34
Movie: Jude, score: 3.3333333333333335
Movie: Nosferatu a Venezia, score: 3.3333333333333335
Movie: Basquiat, score: 3.32
Movie: Bis ans Ende der Welt, score: 3.3142857142857145
Movie: Angel on My Shoulder, score: 3.3125
Movie: 2 Days in the Valley, score: 3.3
Movie: Get on the Bus, score: 3.2941176470588234
Movie: Faust: Love of the Damned, score: 3.2916666666666665
Movie: Fly Away Home, score: 3.282051282051282
Movie

...

Movie: The School of Rock, score: 3.119047619047619
Movie: Must Love Dogs, score: 3.0991379310344827
Movie: Eros, score: 3.0833333333333335
Movie: The Longest Yard, score: 3.0546218487394956
Movie: V for Vendetta, score: 3.05
Movie: Yes Man, score: 3.0454545454545454
Movie: Guess Who, score: 3.0090909090909093
Movie: The Riches, score: 3.0
Movie: Thriller - en grym film, score: 3.0
Movie: Yanks, score: 3.0
Movie: Woman Thou Art Loosed, score: 3.0
Movie: B�se Zellen, score: 3.0
Movie: Smile, score: 3.0
Movie: Seksmisja, score: 3.0
Movie: State Property 2, score: 3.0
Movie: The Bridges of Madison County, score: 3.0
Movie: Jiminy Glick in Lalawood, score: 3.0
Movie: Buying the Cow, score: 3.0
Movie: Airborne, score: 3.0
Movie: Twice in a Lifetime, score: 3.0
Movie: Secuestro express, score: 3.0
Movie: Akasia, score: 3.0
Movie: The Green Berets, score: 2.966666666666667
Movie: My Date with Drew, score: 2.966666666666667
Movie: Pretty Persuasion, score: 2.9615384615384617
Movie: A League of

Movie: Bridge to Terabithia, score: 3.4279279279279278
Movie: Snoopy Come Home, score: 3.426470588235294
Movie: A Guide to Recognizing Your Saints, score: 3.425925925925926
Movie: La tigre e la neve, score: 3.4166666666666665
Movie: The U.S. vs. John Lennon, score: 3.4
Movie: Man cheng jin dai huang jin jia, score: 3.3846153846153846
Movie: Rocky Balboa, score: 3.3728813559322033
Movie: Feast, score: 3.3620689655172415
Movie: Facing the Giants, score: 3.357142857142857
Movie: The History Boys, score: 3.3513513513513513
Movie: SherryBaby, score: 3.34
Movie: Trust the Man, score: 3.3333333333333335
Movie: The Dead Girl, score: 3.323529411764706
Movie: The Holiday, score: 3.3133333333333335
Movie: Les demoiselles de Rochefort, score: 3.3125
Movie: The Guardian, score: 3.3095238095238093
Movie: One Night with the King, score: 3.3
Movie: Death of a President, score: 3.28125
Movie: A Good Year, score: 3.280701754385965
Movie: A Foreign Affair, score: 3.2777777777777777
Movie: Fast Food Natio

Movie: Goy�kiba, score: 3.5
Movie: Ring of Darkness, score: 3.5
Movie: The Boy in the Striped Pyjamas, score: 3.5
Movie: Les amants, score: 3.5
Movie: Gypsy, score: 3.5
Movie: Twilight, score: 3.477272727272727
Movie: Wallace and Gromit in 'A Matter of Loaf and Death', score: 3.3
Movie: Australia, score: 3.176470588235294
Movie: Shrek the Halls, score: 3.1
Movie: Critters 4, score: 3.0
Movie: Soul Men, score: 3.0
Movie: �� maymun, score: 3.0
Movie: Wagon Master, score: 3.0
Movie: The Devil-Doll, score: 3.0
Movie: Cadillac Records, score: 3.0
Movie: Sex Drive, score: 2.8
Movie: Punisher: War Zone, score: 2.75
Movie: Igor, score: 2.5
Movie: Ninja III: The Domination, score: 2.5
Movie: Four Christmases, score: 2.4444444444444446
Movie: Transporter 3, score: 2.230769230769231
Movie: Ace of Hearts, score: 2.0
Movie: Beverly Hills Chihuahua, score: 1.2
Movie: Tuya de hun shi, score: 4.0
Movie: Impulse, score: 4.0
Movie: Zona Zamfirova, score: 4.0
Movie: Manhattan Melodrama, score: 4.0
Movie:

Movie: High Anxiety, score: 3.4776119402985075
Movie: Quantum of Solace, score: 3.4754098360655736
Movie: A Man Called Horse, score: 3.475
Movie: Rumble Fish, score: 3.472972972972973
Movie: The Last Starfighter, score: 3.47244094488189
Movie: The Private Life of Sherlock Holmes, score: 3.4722222222222223
Movie: Factotum, score: 3.4714285714285715
Movie: Vision Quest, score: 3.4705882352941178
Movie: Scarecrow, score: 3.4642857142857144
Movie: Waiting..., score: 3.463855421686747
Movie: Star Wreck: In the Pirkinning, score: 3.463768115942029
Movie: Buffalo Soldiers, score: 3.462686567164179
Movie: Separate Lies, score: 3.4615384615384617
Movie: In Her Shoes, score: 3.460144927536232
Movie: Max, score: 3.46
Movie: The Panic in Needle Park, score: 3.4583333333333335
Movie: Interview, score: 3.4565217391304346
Movie: Love and a Bullet, score: 3.4545454545454546
Movie: De-Lovely, score: 3.4545454545454546
Movie: Sleuth, score: 3.4545454545454546
Movie: Remo Williams: The Adventure Begins, 

Movie: Transporter 2, score: 2.964028776978417
Movie: Prince of Darkness, score: 2.962962962962963
Movie: Drumline, score: 2.9615384615384617
Movie: Code inconnu: R�cit incomplet de divers voyages, score: 2.9615384615384617
Movie: Hitman, score: 2.960526315789474
Movie: The Day the Earth Stood Still, score: 2.96
Movie: Star 80, score: 2.95
Movie: Elizabethtown, score: 2.949275362318841
Movie: Hot Rod, score: 2.9473684210526314
Movie: The Nanny Diaries, score: 2.9473684210526314
Movie: Babylon A.D., score: 2.9423076923076925
Movie: Flight of the Intruder, score: 2.9411764705882355
Movie: Gotcha!, score: 2.9347826086956523
Movie: The Wedding Date, score: 2.9339622641509435
Movie: The In-Laws, score: 2.93
Movie: Go West, score: 2.9285714285714284
Movie: Righteous Kill, score: 2.9285714285714284
Movie: Star Wars: The Clone Wars, score: 2.925925925925926
Movie: The Guru, score: 2.925
Movie: 88 Minutes, score: 2.925
Movie: S.W.A.T., score: 2.924882629107981
Movie: Snowball Express, score: 2.

Recommending the most watched movies (+)
Write a ViewsPredictor that returns the number of views for each movie. This is the recommendation of most viewed movies. For our example, you will get the following:

rtAudienceNumRatings

Movie: The Lord of the Rings: The Fellowship of the Ring, score: 1576 

Movie: The Lord of the Rings: The Two Towers, score: 1528

Movie: The Lord of the Rings: The Return of the King, score: 1457 

Movie: The Silence of the Lambs, score: 1431 

Movie: Shrek, score: 1404

In [18]:
class ViewsPredictor():
    def fit(self, frame):
        self.df = frame.df
    def predict(self):
        self.movies = self.df['id'].to_list()
        dicti = {}
        count=0
        for i in self.movies:
            if ((0 <= count) and (count <= (len(self.movies) - 1))):
                views = self.df.iloc[count]['rtAudienceNumRatings']
                #get rid of /N
                if (repr(views) != repr('\\N')):
                    #print(repr(views))
                    dicti[i] = int(views)
            count = count + 1
        return dicti
    def sort_dict(self, dicti):
        tuples = dicti.items()
        sorted_tuples = sorted(tuples, key=self.sort_second, reverse=True)
        new_dict = {}
        for i in sorted_tuples:
            new_dict[i[0]] = i[1]
        return new_dict
    def sort_second(self, element):
        return element[1]
            
vp = ViewsPredictor()
#movies data created few cells above - mdd
vp.fit(mdd)
dictionary = vp.predict()
#prints predictions for movies with some scores
for item in dictionary: 
    print("Movie: {}, score: {}".format(md.get_title(item), dictionary[item]))

Movie: Toy story, score: 102338
Movie: Jumanji, score: 44587
Movie: Grumpy Old Men, score: 10489
Movie: Waiting to Exhale, score: 5666
Movie: Father of the Bride Part II, score: 13761
Movie: Heat, score: 42785
Movie: Sabrina, score: 12812
Movie: Tom and Huck, score: 2649
Movie: Sudden Death, score: 3626
Movie: GoldenEye, score: 28260
Movie: The American President, score: 8320
Movie: Dracula: Dead and Loving It, score: 10078
Movie: Balto, score: 9195
Movie: Nixon, score: 3256
Movie: Cutthroat Island, score: 3350
Movie: Casino, score: 66463
Movie: Sense and Sensibility, score: 32782
Movie: Four Rooms, score: 14266
Movie: Ace Ventura: When Nature Calls, score: 87306
Movie: Money Train, score: 5263
Movie: Get Shorty, score: 10155
Movie: Copycat, score: 5628
Movie: Ninja Assassin, score: 107023
Movie: Powder, score: 6650
Movie: Leaving Las Vegas, score: 12742
Movie: Othello, score: 2018
Movie: Now and Then, score: 17168
Movie: Persuasion, score: 7968
Movie: La cit� des enfants perdus, score

Movie: With Honors, score: 3484
Movie: Flesh and Bone, score: 851
Movie: Widows' Peak, score: 348
Movie: The Firm, score: 11507
Movie: Free Willy, score: 19451
Movie: Fresh, score: 1823
Movie: The Fugitive, score: 28862
Movie: Geronimo: An American Legend, score: 1686
Movie: A Perfect Getaway, score: 102622
Movie: Getting Even with Dad, score: 2200
Movie: Go Fish, score: 898
Movie: A Good Man in Africa, score: 306
Movie: Guilty as Sin, score: 474
Movie: Hard Target, score: 0
Movie: Heaven & Earth, score: 1458
Movie: Hot Shots! Part Deux, score: 9890
Movie: Live Nude Girls, score: 368
Movie: The Englishman Who Went Up a Hill But Came Down..., score: 3033
Movie: The House of the Spirits, score: 3169
Movie: House Party 3, score: 0
Movie: The Hudsucker Proxy, score: 10291
Movie: I'll Do Anything, score: 481
Movie: In the Army Now, score: 3672
Movie: In the Line of Fire, score: 7831
Movie: In the Name of the Father, score: 10715
Movie: The Inkwell, score: 507
Movie: What's Love Got to Do wi

Movie: It's a Wonderful Life, score: 35280
Movie: Mr. Smith Goes to Washington, score: 869
Movie: Bringing Up Baby, score: 8722
Movie: Penny Serenade, score: 1083
Movie: The Scarlet Letter, score: 3493
Movie: Lady of Burlesque, score: 263
Movie: Of Human Bondage, score: 0
Movie: Angel on My Shoulder, score: 0
Movie: Little Lord Fauntleroy, score: 0
Movie: The Inspector General, score: 0
Movie: Angel and the Badman, score: 0
Movie: The 39 Steps, score: 5603
Movie: A Walk in the Sun, score: 0
Movie: Outlaw, score: 0
Movie: Night of the Living Dead, score: 26113
Movie: The African Queen, score: 6609
Movie: Beat the Devil, score: 793
Movie: Cat on a Hot Tin Roof, score: 6600
Movie: The Last Time I Saw Paris, score: 883
Movie: Meet John Doe, score: 0
Movie: La battaglia di Algeri, score: 4132
Movie: A Farewell to Arms, score: 0
Movie: Huang jia shi jie, score: 0
Movie: On Dangerous Ground, score: 411
Movie: Picnic, score: 632
Movie: Madagascar Skin, score: 295
Movie: The Pompatus of Love, s

Movie: That Darn Cat, score: 2029
Movie: Vegas Vacation, score: 7915
Movie: That Old Feeling, score: 901
Movie: Lost Highway, score: 17451
Movie: Rosewood, score: 1445
Movie: Donnie Brasco, score: 40329
Movie: Booty Call, score: 3264
Movie: Boys Life 2, score: 79
Movie: City of Industry, score: 791
Movie: Children of Men, score: 127695
Movie: Jungle 2 Jungle, score: 9218
Movie: Kama Sutra: A Tale of Love, score: 1793
Movie: Private Parts, score: 0
Movie: Love Jones, score: 4854
Movie: The Saint, score: 11647
Movie: Smilla's Sense of Snow, score: 1224
Movie: Van Helsing, score: 0
Movie: Crash, score: 98746
Movie: The Daytrippers, score: 621
Movie: Liar Liar, score: 93496
Movie: The Quiet Room, score: 0
Movie: Selena, score: 17729
Movie: The Devil's Own, score: 7624
Movie: Cats Don't Dance, score: 2510
Movie: B*A*P*S, score: 1372
Movie: Love and Other Catastrophes, score: 270
Movie: Turbo: A Power Rangers Movie, score: 1946
Movie: Anna Karenina, score: 0
Movie: Double Team, score: 4157
M

Movie: Friday the 13th Part III, score: 0
Movie: Friday the 13th: The Final Chapter, score: 0
Movie: Friday the 13th: A New Beginning, score: 0
Movie: Jason Lives: Friday the 13th Part VI, score: 0
Movie: Friday the 13th Part VII: The New Blood, score: 0
Movie: Friday the 13th Part VIII: Jason Takes Manhattan, score: 9749
Movie: Halloween, score: 79946
Movie: Halloween II, score: 16555
Movie: Halloween III: Season of the Witch, score: 7355
Movie: Halloween 4: The Return of Michael Myers, score: 13190
Movie: Halloween 5, score: 11908
Movie: Prom Night, score: 51435
Movie: Hello Mary Lou: Prom Night II, score: 0
Movie: Prom Night III: The Last Kiss, score: 0
Movie: Prom Night IV: Deliver Us from Evil, score: 0
Movie: Bride of Chucky, score: 20874
Movie: Child's Play 2, score: 13499
Movie: Child's Play 3, score: 12064
Movie: Poltergeist, score: 15891
Movie: Poltergeist II: The Other Side, score: 0
Movie: Poltergeist III, score: 0
Movie: The Exorcist, score: 49968
Movie: Exorcist II: The H

...

Movie: Los olvidados, score: 1531
Movie: Sh�bun, score: 527
Movie: Carrie, score: 23161
Movie: Saikaku ichidai onna, score: 0
Movie: Abbott and Costello Meet Dr. Jekyll and Mr. Hyde, score: 0
Movie: Z Channel: A Magnificent Obsession, score: 0
Movie: Miyamoto Musashi, score: 0
Movie: A Star Is Born, score: 0
Movie: Into the Wild, score: 82515
Movie: Mr. Arkadin, score: 707
Movie: Zoku Miyamoto Musashi: Ichij�ji no kett�, score: 0
Movie: Tuntematon sotilas, score: 0
Movie: Miyamoto Musashi kanketsuhen: kett� Ganry�jima, score: 0
Movie: Teacher's Pet, score: 370
Movie: Ningen no j�ken, score: 0
Movie: Nobi, score: 428
Movie: Pickpocket, score: 1733
Movie: Ukigusa, score: 745
Movie: Onna ga kaidan wo agaru toki, score: 0
Movie: Warui yatsu hodo yoku nemuru, score: 1723
Movie: The Deadly Companions, score: 166
Movie: Ningen no j�ken, score: 0
Movie: Advise & Consent, score: 594
Movie: David and Lisa, score: 0
Movie: Seppuku, score: 1585
Movie: The Music Man, score: 0
Movie: Mutiny on the B

Movie: Mac and Me, score: 2264
Movie: Scorchers, score: 0
Movie: Rosenstrasse, score: 359
Movie: Casque d'or, score: 397
Movie: Touchez pas au grisbi, score: 630
Movie: The MatchMaker, score: 1220
Movie: The Ballad of the Sad Cafe, score: 69
Movie: The Deceivers, score: 252
Movie: Zi hudie, score: 578
Movie: Five Easy Pieces, score: 4148
Movie: El Cid, score: 1234
Movie: Hauru no ugoku shiro, score: 56005
Movie: Such�mub�i, score: 4805
Movie: The Nomi Song, score: 302
Movie: Hitch, score: 189230
Movie: Pooh's Heffalump Movie, score: 2927
Movie: Uncle Nino, score: 109
Movie: Bride & Prejudice, score: 17555
Movie: Constantine, score: 71421
Movie: Son of the Mask, score: 4576
Movie: Because of Winn-Dixie, score: 6908
Movie: Lakposhtha parvaz mikonand, score: 2998
Movie: Beautiful Boxer, score: 879
Movie: Pauly Shore Is Dead, score: 2127
Movie: Bunny Lake Is Missing, score: 715
Movie: Jeremy, score: 0
Movie: Don't Tell Her It's Me, score: 0
Movie: Until September, score: 159
Movie: Night a

Movie: The Ring, score: 146246
Movie: Casanova, score: 29998
Movie: Mrs Henderson Presents, score: 11358
Movie: The White Countess, score: 8910
Movie: Dirty Love, score: 3700
Movie: El esp�ritu de la colmena, score: 0
Movie: Keeping Mum, score: 6183
Movie: Goodnight, Mister Tom, score: 0
Movie: Banshun, score: 1093
Movie: The New World, score: 27173
Movie: 7 Faces of Dr. Lao, score: 0
Movie: The Boys of Baraka, score: 627
Movie: Chinjeolhan geumjassi, score: 23638
Movie: 49th Parallel, score: 0
Movie: Banlieue 13, score: 25880
Movie: BloodRayne, score: 34745
Movie: Hostel, score: 72216
Movie: Grandma's Boy, score: 55076
Movie: Tristan + Isolde, score: 53037
Movie: Glory Road, score: 0
Movie: Last Holiday, score: 31706
Movie: Hoodwinked!, score: 60552
Movie: April's Shower, score: 750
Movie: Underworld: Evolution, score: 111035
Movie: Looking for Comedy in the Muslim World, score: 4171
Movie: Tini zabutykh predkiv, score: 312
Movie: Cul-de-sac, score: 1039
Movie: The Dunwich Horror, sco

Movie: How the Grinch Stole Christmas, score: 34926
Movie: Goya's Ghosts, score: 20033
Movie: Perfect Stranger, score: 43661
Movie: Disturbia, score: 701373
Movie: Pathfinder, score: 39732
Movie: Aqua Teen Hunger Force Colon Movie Film for The..., score: 12293
Movie: Puccini for Beginners, score: 328
Movie: Yihe yuan, score: 653
Movie: La m�me, score: 33
Movie: The Contract, score: 0
Movie: Fracture, score: 129211
Movie: Big Nothing, score: 0
Movie: Vacancy, score: 51558
Movie: Benny's Video, score: 0
Movie: In the Land of Women, score: 37584
Movie: Mr. Bean's Holiday, score: 95541
Movie: The Invisible, score: 49604
Movie: Kickin It Old Skool, score: 0
Movie: The Condemned, score: 27981
Movie: Spider-Man 3, score: 1192712
Movie: Lucky You, score: 26515
Movie: 21, score: 0
Movie: Year of the Dog, score: 6288
Movie: Fast Track, score: 24933
Movie: Papurika, score: 13141
Movie: Red Road, score: 3304
Movie: Dnevnoy dozor, score: 13807
Movie: This Is England, score: 25557
Movie: Away from H

Movie: Finding Amanda, score: 997
Movie: Kit Kittredge: An American Girl, score: 19095
Movie: Gonzo: The Life and Work of Dr. Hunter S. Thompson, score: 7310
Movie: The Wackness, score: 8603
Movie: Up the Yangtze, score: 954
Movie: Along Came Jones, score: 0
Movie: Sleepwalking, score: 8038
Movie: Encounters at the End of the World, score: 4183
Movie: Mauvais sang, score: 0
Movie: Standard Operating Procedure, score: 1257
Movie: Zombie Strippers!, score: 3970
Movie: Battle for Haditha, score: 1153
Movie: Aleksandra, score: 547
Movie: Transylvania, score: 788
Movie: Mamma Mia!, score: 126612
Movie: The Game, score: 23139
Movie: Erogotoshi-tachi yori: Jinruigaku ny�mon, score: 0
Movie: Kabluey, score: 1396
Movie: The Stone Angel, score: 966
Movie: Rogue, score: 3997
Movie: Waga seishun ni kuinashi, score: 0
Movie: It's the Great Pumpkin, Charlie Brown, score: 0
Movie: Katyn, score: 3505
Movie: Journey to the Center of the Earth, score: 38002
Movie: Meet Dave, score: 22234
Movie: Kataude 

Predicting scores with similarity between products (+)

Write an ItemBasedPredictor class that accepts two parameters in the constructor: min_values and threshold (the default values of both should be 0). 
Based on the similarity between the products, the class should calculate the expected product rating for the selected user. Calculate the similarity with the corrected cosine distance.

In class, implement the fit, predict, and similarity(self, p1, p2) methods, which returns the similarity between the p1 and p2 products. If the calculated similarity between the two products is less than the threshold or if we have less than min_values users who rated both movies, the similarity between the two products should be 0. I suggest calculating all similarities in the fit method and only using them in predict and fit.

In [3]:
import math
import numpy as np
from scipy import spatial

class ItemBasedPredictor():
    def __init__(self, min_values, threshold):
        self.min_values = min_values
        self.threshold = threshold
        #NUMBER OF USERS TO CALCULATE SIMILARITY WITH(LOWER TO REDUCE COMP. TIME)
        self.num_users = 1000
    def fit(self, frame):
        #get df with certain min_ratings filter condition
        self.df = frame.df
        self.movies = list(self.df.movieID.unique())
        self.users = list(self.df.userID.unique())
    #If the similarity < threshold || min_values > users who rated both movies -> similarity = 0
    #returns dict movieID:user_id's predicted rating
    def predict(self, user_id, n, rec_seen):
        return "lol"
    def similar_items(self, movie_id, n):
        #self.df so vse ocene
        mean = self.df.groupby(['userID'],as_index = False, sort = False)\
                .mean().rename(columns = {"userID": "userID", "rating": "rating_mean"})
        ratings = pd.merge(self.df, mean, on='userID', how="left", sort = "False")
        ratings['norm_rating'] = ratings["rating"] - ratings["rating_mean"]
        #print(type(ratings['movieID_x']))
        
        matrix1 = pd.DataFrame({"user":ratings['userID'],
                              "movie":ratings['movieID_x'],
                              "rating":ratings['norm_rating']})
        #tuki mogoce menjaj vrstice in stolpce
        matrix = matrix1.pivot_table(index='movie', columns='user', values = 'rating').fillna(0)
        #print(matrix)
        the_movie = matrix.loc[movie_id]
        #print(type(mov.to_numpy()))
        #print(mov.to_numpy())
        
        all_users = matrix.values
        denom_1 = np.sqrt(sum([np.square(x) for x in the_movie]))
        
        cos_sim = [(316,1)]
        i = 1
        for user in all_users[1:]:
            numer = [x*y for x,y in zip(the_movie, user)]
            denom_2 = np.sqrt(sum([np.square(x) for x in user]))
            cos = sum(numer) / (denom_1 * denom_2)
            cos_sim.append((matrix.index[i], cos))
            i += 1
            
        cos_sim.sort(key=lambda x: x[1], reverse=True)
        sim_movies = cos_sim[0:n]
        #print(sim_movies)
        
        #print(top10movies)                           
        return sim_movies
    #Most similar movies (+)
    #List 20 pairs of the most similar movies.
    def most_similar_movies(self, n):
        #self.df so vse ocene
        mean = self.df.groupby(['userID'],as_index = False, sort = False)\
                .mean().rename(columns = {"userID": "userID", "rating": "rating_mean"})
        ratings = pd.merge(self.df, mean, on='userID', how="left", sort = "False")
        ratings['norm_rating'] = ratings["rating"] - ratings["rating_mean"]
        #print(type(ratings['movieID_x']))
        
        matrix1 = pd.DataFrame({"user":ratings['userID'],
                              "movie":ratings['movieID_x'],
                              "rating":ratings['norm_rating']})
        #tuki mogoce menjaj vrstice in stolpce
        matrix = matrix1.pivot_table(index='movie', columns='user', values = 'rating').fillna(0)
        #print(matrix)
        
        similarities = []
        m_l = ratings['movieID_x']
        print(m_l)
        for ind in m_l[:5]:
            #print(ind)
            the_movie = matrix.loc[ind]
            #print(the_movie)

            all_users = matrix.values
            denom_1 = np.sqrt(sum([np.square(x) for x in the_movie]))

            i = 1
            for user in all_users[1:]:
                numer = [x*y for x,y in zip(the_movie, user)]
                denom_2 = np.sqrt(sum([np.square(x) for x in user]))
                cos = sum(numer) / (denom_1 * denom_2)
                #print(cos)
                if cos < 0.98:
                    similarities.append((the_movie, (matrix.index[i], cos)))
                i += 1
        sim_all = similarities.sort(key=lambda x: x[1][1], reverse=True)
        sim_movies = sim_all[0:n]                          
        return "sim_movies"
        
                                                                                                  

In [4]:
md_ibp = MovieData('data/movies.dat')
uim_2_ibp = UserItemData('data/user_ratedmovies.dat')
ibp = ItemBasedPredictor(0, 0)
ibp.fit(uim_2_ibp)
#print(ibp.movies)
#recommender1 = Recommender(ibp)
#recommender1.fit(uim_2_ibp)

#rec_items_ibp = recommender1.recommend(78, 15, rec_seen=False)
#for idmovie, val in rec_items_ibp.items():
 #   print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

#Most similar movies (+)
#List 20 pairs of the most similar movies.
ibp.most_similar_movies(3)
#most_similar_movies = ibp.most_similar_movies(3)
#for movie1, (movie2, val) in most_similar_movies:
 #   print("Movie1: {}, Movie2: {} ,score: {}".format(md.get_title(movie1), md.get_title(movie2), val))
    
#Recommendation based on the currently viewed content (+)    
#print(ibp.similar_items(1580, 10))
#rec_items = ibp.similar_items(4993, 10)
#for idmovie, val in rec_items:
 #   print("Movie: {}, score: {}".format(md.get_title(idmovie), val))






0             3
1            32
2           110
3           160
4           163
          ...  
855593    44555
855594    46578
855595    48516
855596    61075
855597    62049
Name: movieID_x, Length: 855598, dtype: int64
3
0.095972978031761
1.0
0.14974176278068177
0.21300902268109564
-0.07655707675705516
0.10669835842186072
0.07555125231727325
0.057030786408842855
0.03092530125548827
0.009857188095937492
0.1426680649549925
0.15715073017047185
-0.06742391491763326
0.06092323927916633
-0.06346424541645065
-0.09563419973591995
0.07948697170108412
0.17801319229498636
0.07294128611123986
-0.0031842554033802402
-0.008903692902516807
0.0779729829098723
0.06381435828742381
-0.05815280796055717
-0.06765581433322408
-0.03853113597652394
-0.07028889339806288
-0.0592052813274829
-0.008381487257194155
0.039493411684983236
-0.11429226396547111
0.0
-0.005494771685695066
0.09377303967010983
-0.10457599885585503
0.0
0.16303123888077684
-0.001231205064100475
-0.006478338776697323
-0.06430485446214612
-

-0.02699815952540519
0.0
-0.006692021272963288
-0.06269932995910286
-0.05570550072764392
0.021526898049751785
-0.07004028299941013
0.0782716033290638
-0.048691935647863206
0.007504923016483215
0.04074146247553445
-0.06093044608213462
0.09989712851572888
-0.06332697157644225
-0.02896275478605587
-0.025793310421426758
-0.035844941790729334
-0.11598842890255821
-0.08843091195992976
-0.029733866950488515
-0.11573064608036664
-0.11035896977153214
-0.09785311477453708
-0.050678625834555006
-0.10714727322818415
-0.0276520352028053
0.03898019006030636
-0.12185880550334657
-0.11208608761451117
0.09057856340509436
0.023365059197005808
-0.004189512956198091
-0.04910634673729449
-0.041401683396857424
-0.03408078742469091
-0.12888961396897386
-0.04363481582014808
-0.03785221950675801
0.0709939371746753
0.07149027554503129
-0.021526898049751785
-0.16283891643108964
-0.13517087982060066
0.013193457606494114
0.020887749516364247
-0.014480953458872204
-0.031577947020931606
0.15484773069760668
0.0595225

-0.03280804213308346
-0.06952150342542017
0.006064094747688403
-0.02616869451029455
-0.01578741533546112
0.0038199205904436723
-0.0015786187488327419
-0.1338690771941402
0.09650444781958037
-0.015085709398454439
-0.03893965304530811
0.04775097249742436
-0.006534730720892903
-0.10706748523134636
0.0051154353632687395
0.0176161399302534
0.0
-0.08613241008850457
0.048461087818818055
0.06440653584162376
0.0
-0.06380008861351849
0.041227654598255975
-0.12476857964757528
-0.0012950195500124936
0.01663918592199347
-0.032175936339328345
-0.05854415868460148
0.0683256357063493
0.07475536764475492
0.07599617168839437
-0.035392813023025396
0.040487822959657346
-0.02564722780685362
0.011451217804747367
-0.011725310348677057
-0.04602015182162156
-0.08476095287688457
-0.0708235819852487
0.03361498157198302
0.009125948925989898
-0.09926567724425495
0.007054374476394993
0.006785517116280598
0.0800654765686316
-0.10762787473140212
-0.025302754783545506
-0.037213287959247596
0.0034390749515972193
0.0780

0.041770264838025005
-0.02235853588509125
0.05189817399412458
0.011390595444467664
-0.0020654409604200837
-0.0023183220202523972
-0.0989558970978793
0.005300876148520918
-0.050774904202888785
-0.12281711636879954
-0.03317050940927942
0.08287474695206432
-0.02380102592188505
0.06070086080925628
0.0840349847789482
0.0
-0.00010388917691996793
-0.11680715345634815
0.032059664332177044
0.017159240386084195
-0.03833487952407138
0.0
0.0
0.027685640120154793
-0.07083015040457596
-0.14291692112394427
-0.01418416521811104
-0.013255794380103925
-0.14854318784343937
0.013787111611860704
-0.11666399936458607
-0.023037179746135738
-0.02879057389361667
-0.03063530304174351
-0.014160883823955549
0.009822876347918036
-0.0011634428415239126
0.02310678255112322
0.0247475413802292
0.011669674705208417
-0.011590161832797687
0.037629692362275514
0.0
0.0
0.0261231482836122
0.0
-0.0008718515550096255
-0.053642857833562686
-0.03217000106962321
0.021575380358313738
-0.04437479124832983
0.0
-0.010154084326910937

-0.06345667146550667
0.1765632741979864
0.000949456535402687
-0.022412294066516163
-0.004972213150976396
0.0
0.06582738195458283
0.021811161228279398
-0.08373620973199927
0.05287227505624803
0.05842116706587301
0.029966081368420893
-0.012441409768824489
0.08582719238268226
-0.023015572689986663
-0.005053535428290043
-0.10459262898111324
0.044075933682552894
0.03176286119852549
0.09916313067816374
0.11947414599506047
-0.06569721895702219
0.059723061915086494
0.012559579802884708
0.19070176012841203
-0.004622192599677933
-0.07232625638628311
0.020544779496718235
-0.0593799929859282
-0.08019085220760064
-0.10148681006113874
-0.11346149619007365
-0.07049995298048874
0.04127960300056216
0.028391086025996482
0.08480348076442457
0.06132976877122259
0.0
0.043390615813034224
-0.08603508509147675
0.03531818638760104
0.01035545385248393
-0.06794198039530282
-0.003534166103418534
0.0222073590265742
0.012043901900135686
-0.0017151239497884769
0.17577228202809914
0.026162104138175067
0.0096683665533

...

0.006561131669477151
-0.020751274100793665
-0.04334883441237299
0.0
-0.07932904794403688
-0.009091046780227804
-0.009385953485716837
-0.004980088428276076
-0.0632532945690083
-0.05279424776722324
0.0025642114123623537
-0.009238501006288405
-0.11545473332745845
-0.09254340940444578
-0.019528025350275667
0.0
0.005151911005379289
0.0
-0.014865999199416145
0.0161355017632237
-0.07850239288061296
-0.0033806266765115623
-0.010228558633537081
-0.04114739719995594
-0.02483683118379878
0.01333616417619523
-0.032335773311321515
-0.07980819313972656
0.0
0.012153080215590972
-0.029655125815064548
-0.024706732952699095
-0.08725475290012646
-0.0015583365630977236
0.0
-0.048066832378183336
-0.0081321173230854
0.00399618450770142
-0.019098242007326106
-0.0022567954348348203
0.019543898092408092
-0.020363465104115165
0.0
-0.010183544294255885
-0.040835996691314394
0.0
-0.05339925149637441
0.02702558603634564
0.07089331851008387
-0.022790539811345115
0.02775486618535316
0.02527303545053389
-0.0130392545

-0.06687261549371067
0.0
-0.03241299563165931
-0.011787150784069469
0.024414111134912313
-0.03586642347409457
-0.09973590629482362
-0.05012247697015317
0.013042243528936475
0.08507375732663558
-0.014314596440358316
0.036069103850390465
-0.03602632055225877
0.011224253492488909
-0.024067305870013125
-0.01962586803400489
0.009127986600193337
-0.010732455624027836
-0.04026420816018631
0.044626785174855625
-0.003048244891671312
0.029068605149309635
-0.002904763026969437
0.10889601437046832
-0.027633055895784197
-0.050258493770666664
0.0027206920640026493
-0.05540817690022455
0.14743701364513184
0.016720496346247494
0.011457728071566335
-0.0046794992979157625
-0.0600838843021148
0.003125382422917335
-0.07273008473281528
-0.005836576219874007
0.030652408596448834
0.0
0.12037991239720167
-0.014804541082013477
0.0
0.0019895970894889294
-0.04903373007682565
-0.02216235786412075
-0.021024397407934048
-0.07197014999946454
-0.05480968429330501
0.0013815136185239473
-0.04956929605220886
-0.01467973

0.004783996542919996
-0.00026787308292170814
-0.004364502712740728
0.01892146713853558
0.002971670442666518
-0.06976501819754984
-0.021743478736995104
0.02275001603535845
0.012083741109341368
-0.053532531522647586
0.016909961071778095
0.02407645394042269
-0.01225826612568386
-0.006357053395037596
0.035278630556959636
0.06567053033618706
-0.031203454893919586
0.03925183584282821
0.04243550930695593
-0.020847722140077476
-0.0020959708347646536
-0.02134507337364253
-0.007283196016950988
0.03981130428459793
0.042831453541760066
-0.006691625705214258
-0.010865036567268097
0.019637456924996497
0.02077506406679584
-0.006031400307733168
-0.007353213670194049
-0.02948556902515853
-0.00023901160107629726
-0.025304955347487015
-0.012007117508872005
-0.048283013889867776
-0.0056675358664703135
0.008373030771791964
0.02723041223700054
-0.015723762571274565
0.003958774914618889
0.02455037618004863
-0.023113476765172593
-0.040173221392178465
-0.011798083476099851
0.0
-0.04233549810534827
-0.049400748

-0.006818292404812877
0.03936446115045932
-0.02993593357176928
0.0
0.0276388094836903
-0.012509825823207376
0.10896267778655032
-0.01841541071416008
0.013916076258923755
-0.021626658139316222
0.010851245009283726
0.013426833295907301
0.0
-0.008403495662540292
-0.02678131981232024
0.010475985760808397
-0.010095116973185091
0.020528996496664637
-0.03813290438507497
-0.009347478722336836
-0.0020707251592972232
0.015462367315415965
0.039621930771587696
-0.009279142558790877
0.050910346162145174
0.046248970028021835
0.010027314211419669
-0.030032800612487684
0.05725989447912992
0.025075014003430925
-0.025768822861705367
-0.07816963063526333
0.021537408575346275
-0.009246321177286998
0.0
0.012734105375018004
0.037658454609630826
0.04753363291953566
-0.023767370164895347
0.015117420095483221
0.02301948269545263
0.0037607616458258677
-0.05536230351394439
-0.029020277105183512
0.005074577651042391
0.06310282955518198
-0.0059915645799621985
-0.015134855398905238
-0.010617036046966875
0.026260052

TypeError: 'NoneType' object is not subscriptable

In [15]:
 def create_rates_table(self):
        #all distinct movies
        am = self.df['movieID'].to_list()
        all_movies = list(dict.fromkeys(am))
        #all distinct users
        au = self.df['userID'].to_list()
        all_users = list(dict.fromkeys(au))
        
        table = pd.DataFrame([], index = all_users, columns = all_movies)
        for row in table.iterrows():
            for col in table.columns:
                #movieID
                m_id = self.df.loc[(self.df['movieID'] == col) & (self.df['userID'] == row)].reset_index()
                movie = int(m_id.loc[0]['movieID'])
                #userID
                user = int(m_id.loc[0]['userID'])
                #rate user gave to movie
                rate = float(m_id.loc[0]['rating'])
                table.set_value(row, col , rate )
        return table

In [201]:
d = {1: (3,"fd"), 2: (4,"ola")}
print(d.items())
print(d[1][1])

dict_items([(1, (3, 'fd')), (2, (4, 'ola'))])
fd


In [34]:
d = {1:3}
if (2 in d.keys):
    print("lol")

TypeError: argument of type 'builtin_function_or_method' is not iterable